In [0]:
dbutils.widgets.text("catalog",'')

In [0]:
catalog = dbutils.widgets.get("catalog")

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS autoloader_tempDB

In [0]:
from pyspark.sql.functions import * 

file_path = f'/Volumes/{catalog}/conformed/autoloader_demo'
target_path = f'/Volumes/{catalog}/conformed/autoloader_result'
checkpoint_path = f'/Volumes/{catalog}/conformed/checkpoint_forautoloader_demo'
table_name = 'PiotrT_Demo.AutoLoaderDemo'  #Hive metastore database 
# This works in cloud like azure adls, aws s3
(spark.readStream
    .format("cloudFiles") 
    .option("cloudFiles.format", 'csv')
    .option("cloudFiles.schemaLocation", checkpoint_path)
    .load(file_path)
    .select("*", col("_metadata.file_path").alias("source_path"), current_timestamp().alias("processing_time"))
    .writeStrem
    .option("checkpointLocation", checkpoint_path)
    .trigger(processingTime="1 seconds")
    .toTable(table_name))

In [0]:
df = spark.read.format('json')\
    .load(f'/Volumes/{catalog}/big_mart/mart_csv/minifigs.json')

display(df)

In [0]:
from pyspark.sql.functions import explode
df2= df.withColumn('explodedArray', explode(df.results))

In [0]:
df2.createOrReplaceTempView("minifigs_view")

In [0]:
%sql
SELECT 
 *
FROM 
 minifigs_view

In [0]:
%sql
WITH MINIFIGS_CTE AS ( 
SELECT 
    explodedArray.last_modified_dt as LastModifiedOrignal,
    cast(explodedArray.last_modified_dt as date) as LastModifiedDate,
    explodedArray.name AS Name,
    CAST(explodedArray.num_parts AS INT) as NumberOfParts,
    explodedArray.set_img_url,
    explodedArray.set_num,
    explodedArray.set_url
FROM
  minifigs_view
)

SELECT * FROM MINIFIGS_CTE



In [0]:
_sqldf.write.format('delta').save(f'/Volumes/{catalog}/silver/silvervolume')

In [0]:
delta_df = spark.read.format('delta').load(f'/Volumes/{catalog}/silver/silvervolume')
display(delta_df)

In [0]:
%sql
CREATE DATABASE Minifigs


In [0]:
%sql
CREATE TABLE Minifigs.Minifigs_table
-- LOCATION
AS
SELECT 
    explodedArray.last_modified_dt as LastModifiedOrignal,
    cast(explodedArray.last_modified_dt as date) as LastModifiedDate,
    explodedArray.name AS Name,
    CAST(explodedArray.num_parts AS INT) as NumberOfParts,
    explodedArray.set_img_url,
    explodedArray.set_num,
    explodedArray.set_url
FROM
  minifigs_view

In [0]:
%sql
SELECT * FROM Minifigs.Minifigs_table